In [1]:
import sqlite3
import pandas as pd  
import talib
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
import akshare as ak
import numpy as np
%matplotlib inline   

#正常显示画图时出现的中文和负号
from pylab import mpl

mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

In [2]:
data = r"G:\fund\dbdata\stock_2018_daily\stock_2018.db"

## 财务数据构成

财务三张表指的是资产负债表、利润表以及现金流量表，具体概念分别如下：


　　1、资产负债表，资产负债表亦称财务状况表，表示企业在一定日期（通常为各会计期末）的财务状况（即资产、负债和所有者权益的状况）的主要会计报表。

　　2、利润表，利润表是反映企业在一定会计期间经营成果的报表。由于它反映的是某一期间的情况，所以又被称为动态报表。利润表也被称为损益表、收益表。

　　3、现金流量表，是反映在一固定期间（通常是每月或每季）内，企业的现金（包含银行存款）的增减变动情形。

1、资产负债表（balance sheet）：是总括地反映会计主体在特定日期（如年末、季末、月末）财务况的报表。

2、损益表（income statement）：也称收益表、损益表，它是总括反映企业在某一会计期间（如年度、季度、月份）内经营及其分配（或弥补）情况的一种会计报表。

3、现金流量表（statement of cash flow）：反映企业现金流量的来龙去脉，当中分为经营活动、投资活动及筹资活动三部份。

## akshare中提供的财务数据

* akshare 中提供的财务数据更多样，并且能实时，但是很多函数总是存在变动，官网文档有时更新不及时，建议多用help了解变化，通过百度也常常能定位到作者知乎上的最新文档

https://www.zhihu.com/people/bjck

* akshare的财务数据

### “akshare股票数据”——“基本面数据”——“财务报表”（这里主要提供某个股票所有历史财务数据）


In [3]:
help(ak.stock_financial_report_sina)

Help on function stock_financial_report_sina in module akshare.stock_fundamental.stock_finance:

stock_financial_report_sina(stock: str = 'sh600600', symbol: str = '资产负债表') -> pandas.core.frame.DataFrame
    新浪财经-财务报表-三大报表
    https://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinanceSummary/stockid/600600/displaytype/4.phtml?source=fzb&qq-pf-to=pcqq.group
    :param stock: 股票代码
    :type stock: str
    :param symbol: choice of {"资产负债表", "利润表", "现金流量表"}
    :type symbol: str
    :return: 新浪财经-财务报表-三大报表
    :rtype: pandas.DataFrame



In [6]:
byd_zcfz = ak.stock_financial_report_sina(stock="002594", symbol="资产负债表")
byd_lrb = ak.stock_financial_report_sina(stock="002594", symbol="利润表")
byd_xjllb=ak.stock_financial_report_sina(stock="002594", symbol="现金流量表")

In [7]:
byd_zcfz

,报告日,流动资产,货币资金,结算备付金,拆出资金,交易性金融资产,买入返售金融资产,衍生金融资产,应收票据及应收账款,应收票据,...,归属于母公司股东权益合计,少数股东权益,所有者权益(或股东权益)合计,负债和所有者权益(或股东权益)总计,数据源,是否审计,公告日期,币种,类型,更新日期
0,20240930,NaN,66319375000.0,NaN,NaN,25415146000.0,200984000.0,NaN,79443781000.0,NaN,...,155462393000.0,13340334000.0,168802727000.0,764259515000.0,定期报告,未审计,20241031,CNY,合并期末,2024-10-30T19:50:05
1,20240630,NaN,53053301000.0,NaN,NaN,17388054000.0,1760305000.0,NaN,71814516000.0,NaN,...,142787204000.0,11823874000.0,154611078000.0,686244710000.0,定期报告,未审计,20240829,CNY,合并期末,2024-08-28T22:40:06
2,20240331,NaN,86796529000.0,NaN,NaN,9384994000.0,NaN,NaN,61265729000.0,NaN,...,143090246000.0,11856789000.0,154947035000.0,677783621000.0,定期报告,未审计,20240430,CNY,合并期末,2024-04-29T20:10:13
3,20231231,NaN,109094408000.0,NaN,NaN,9562550000.0,NaN,NaN,61866019000.0,NaN,...,138810065000.0,11652048000.0,150462113000.0,679547670000.0,定期报告,是,20241031,CNY,合并期末,2024-03-26T21:20:04
4,20230930,NaN,55759753000.0,NaN,NaN,10611806000.0,NaN,NaN,52970472000.0,NaN,...,129855730000.0,11211127000.0,141066857000.0,623283496000.0,定期报告,未审计,20231031,CNY,合并期末,2023-10-30T18:45:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,20061231,NaN,1701080000.0,NaN,NaN,1320000.0,NaN,NaN,2994400000.0,656460000.0,...,4940350000.0,75180000.0,5015530000.0,16030800000.0,募集说明书,是,20070619,CNY,合并期末,2020-03-13T15:29:48
62,20051231,NaN,730180000.0,NaN,NaN,3730000.0,NaN,NaN,2123530000.0,257340000.0,...,3930340000.0,64440000.0,3994780000.0,10949220000.0,募集说明书,是,20070619,CNY,合并期末,2020-03-13T15:29:48
63,20041231,NaN,706410000.0,NaN,NaN,NaN,NaN,NaN,1910638000.0,113385000.0,...,3867721000.0,59198000.0,3926919000.0,8627087000.0,其他,是,20060717,CNY,合并期末,2020-03-13T15:29:48
64,20031231,NaN,518491000.0,NaN,NaN,NaN,NaN,NaN,1493362000.0,92548000.0,...,3246369000.0,153529000.0,3399898000.0,5544029000.0,其他,是,20060717,CNY,合并期末,2020-03-13T15:29:48


In [8]:
byd_lrb

,报告日,营业总收入,营业收入,利息收入,已赚保费,手续费及佣金收入,房地产销售收入,其他业务收入,营业总成本,营业成本,...,归属于母公司所有者的综合收益总额,归属于少数股东的综合收益总额,基本每股收益,稀释每股收益,数据源,是否审计,公告日期,币种,类型,更新日期
0,20240930,502251312000.0,502251312000.0,NaN,NaN,NaN,NaN,NaN,478028764000.0,397953563000.0,...,25196914000.0,1001071000.0,8.68,8.68,定期报告,未审计,20241031,CNY,合并期末,2024-10-30T21:52:02
1,20240630,301126713000.0,301126713000.0,NaN,NaN,NaN,NaN,NaN,287855895000.0,240859982000.0,...,13115818000.0,485421000.0,4.68,4.68,定期报告,未审计,20240829,CNY,合并期末,2024-08-29T00:57:03
2,20240331,124944397000.0,124944397000.0,NaN,NaN,NaN,NaN,NaN,120733401000.0,97603387000.0,...,4143094000.0,202348000.0,1.57,1.57,定期报告,未审计,20240430,CNY,合并期末,2024-04-29T20:13:37
3,20231231,602315354000.0,602315354000.0,NaN,NaN,NaN,NaN,NaN,567681132000.0,480558350000.0,...,30216142000.0,1311956000.0,10.32,10.32,定期报告,是,20240327,CNY,合并期末,2024-03-26T22:57:02
4,20230930,422274838000.0,422274838000.0,NaN,NaN,NaN,NaN,NaN,397395929000.0,338726669000.0,...,21423611000.0,967485000.0,7.35,7.35,定期报告,未审计,20241031,CNY,合并期末,2023-10-30T20:07:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,20061231,13128190000.0,13098240000.0,NaN,NaN,NaN,NaN,NaN,12044530000.0,10200740000.0,...,NaN,NaN,NaN,NaN,募集说明书,是,20070619,CNY,合并期末,2020-03-13T16:00:40
65,20051231,6555696000.0,6529023000.0,NaN,NaN,NaN,NaN,NaN,6108060000.0,5043785000.0,...,NaN,NaN,NaN,NaN,募集说明书,是,20070619,CNY,合并期末,2020-03-13T16:00:40
66,20041231,6451608000.0,6441869000.0,NaN,NaN,NaN,NaN,NaN,5484823000.0,4720863000.0,...,NaN,NaN,NaN,NaN,其他,是,20060717,CNY,合并期末,2020-03-13T16:00:40
67,20031231,4114367000.0,4076006000.0,NaN,NaN,NaN,NaN,NaN,3194828000.0,2743324000.0,...,NaN,NaN,NaN,NaN,其他,是,20060717,CNY,合并期末,2020-03-13T16:00:40


In [9]:
byd_xjllb

,报告日,经营活动产生的现金流量,销售商品、提供劳务收到的现金,客户存款和同业存放款项净增加额,向中央银行借款净增加额,向其他金融机构拆入资金净增加额,收到原保险合同保费取得的现金,收到再保险业务现金净额,保户储金及投资款净增加额,处置交易性金融资产净增加额,...,现金的期初余额,现金等价物的期末余额,现金等价物的期初余额,期末现金及现金等价物余额,数据源,是否审计,公告日期,币种,类型,更新日期
0,20240930,NaN,501573341000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,65809661000.0,定期报告,未审计,20241031,CNY,合并期末,2024-10-30T19:50:07
1,20240630,NaN,297083766000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,108511745000.0,NaN,NaN,54226572000.0,定期报告,未审计,20240829,CNY,合并期末,2024-08-28T23:05:10
2,20240331,NaN,124156838000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,86178748000.0,定期报告,未审计,20240430,CNY,合并期末,2024-04-29T20:10:12
3,20231231,NaN,572704798000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,51182457000.0,NaN,NaN,108511745000.0,定期报告,是,20240327,CNY,合并期末,2024-03-26T21:25:05
4,20230930,NaN,396455740000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,55265993000.0,定期报告,未审计,20241031,CNY,合并期末,2023-10-30T19:20:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,20061231,NaN,12707928000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,730179000.0,NaN,NaN,1701077000.0,募集说明书,是,20070619,CNY,合并期末,2020-03-13T14:58:48
65,20051231,NaN,7944214000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,706410000.0,NaN,NaN,730179000.0,募集说明书,是,20070619,CNY,合并期末,2020-03-13T14:58:48
66,20041231,NaN,6247631000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,498813000.0,NaN,NaN,599394000.0,其他,是,20060717,CNY,合并期末,2020-03-13T14:58:48
67,20031231,NaN,3065540000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1240156000.0,NaN,NaN,671600000.0,其他,是,20060717,CNY,合并期末,2020-03-13T14:58:48


In [10]:
byd_cwzb = ak.stock_financial_analysis_indicator(symbol="002594")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [12]:
help(ak.stock_financial_analysis_indicator)

Help on function stock_financial_analysis_indicator in module akshare.stock_fundamental.stock_finance:

stock_financial_analysis_indicator(symbol: str = '600004', start_year: str = '1900') -> pandas.core.frame.DataFrame
    新浪财经-财务分析-财务指标
    https://money.finance.sina.com.cn/corp/go.php/vFD_FinancialGuideLine/stockid/600004/ctrl/2019/displaytype/4.phtml
    :param symbol: 股票代码
    :type symbol: str
    :param start_year: 开始年份
    :type start_year: str
    :return: 新浪财经-财务分析-财务指标
    :rtype: pandas.DataFrame



In [13]:
stock_zcfz_em_df = ak.stock_zcfz_em(date="20220331")
stock_lrb_em_df = ak.stock_lrb_em(date="20220331")
stock_xjll_em_df = ak.stock_xjll_em(date="20220331")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

akshare的财务指标 接口不稳，提供的是横截面的指标

In [14]:
# 使用收集好的数据

In [15]:
conn=sqlite3.connect(data)

In [16]:
xjllb=pd.read_sql("select * from xjllb",con=conn)
lrb=pd.read_sql("select * from lrb",con=conn)
zcfzb=pd.read_sql("select * from zcfzb",con=conn)
cwzbsj=pd.read_sql("select * from cwzbsj",con=conn)
stock_daily=pd.read_sql("select * from stock_daily where 股票代码<'003000.SZ'",con=conn)

In [17]:
stock_daily

,index,交易日期,股票代码,股票简称,开盘价,最高价,最低价,收盘价,成交量(手),成交额(千元),...,市盈率(动态),市净率,市销率,市销率(TTM),股息率(%),股息率(TTM)(%),总股本(万股),流通股本(万股),总市值(万元),流通市值(万元)
0,6323,20180102,000001.SZ,平安银行,1419.2252,1480.8844,1416.0359,1456.4333,2081592.55,2856543.822,...,9.21,1.1871,2.1839,2.2280,1.1533,1.1533,1.717041e+06,1.691799e+06,2.352346e+07,2.317765e+07
1,6324,20180103,000001.SZ,平安银行,1459.6226,1473.4427,1403.2788,1417.0990,2962498.38,4006220.766,...,8.96,1.1550,2.1249,2.1678,1.1853,1.1853,1.717041e+06,1.691799e+06,2.288816e+07,2.255168e+07
2,6325,20180104,000001.SZ,平安银行,1416.0359,1421.3513,1395.8372,1408.5942,1854509.48,2454543.516,...,8.91,1.1481,2.1121,2.1548,1.1925,1.1925,1.717041e+06,1.691799e+06,2.275080e+07,2.241634e+07
3,6326,20180105,000001.SZ,平安银行,1404.3419,1419.2252,1397.9634,1413.9097,1210312.72,1603289.517,...,8.94,1.1524,2.1201,2.1630,1.1880,1.1880,1.717041e+06,1.691799e+06,2.283665e+07,2.250093e+07
4,6327,20180108,000001.SZ,平安银行,1408.5942,1412.8466,1367.1337,1377.7646,2158620.81,2806099.169,...,8.71,1.1230,2.0659,2.1077,1.2191,1.2191,1.717041e+06,1.691799e+06,2.225285e+07,2.192571e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730975,589,20230213,002999.SZ,天禾股份,14.3820,14.6696,14.3676,14.4827,173284.55,174254.521,...,29.29,3.0625,0.2692,0.2123,0.9930,0.9930,3.475920e+04,2.177714e+04,3.500251e+05,2.192958e+05
1730976,590,20230214,002999.SZ,天禾股份,14.5977,14.6121,14.1087,14.1231,250697.29,248112.457,...,28.57,2.9865,0.2626,0.2070,1.0183,1.0183,3.475920e+04,2.177714e+04,3.413353e+05,2.138515e+05
1730977,591,20230215,002999.SZ,天禾股份,14.1375,14.1663,13.8786,13.9074,174541.09,169258.922,...,28.13,2.9409,0.2585,0.2039,1.0341,1.0341,3.475920e+04,2.177714e+04,3.361215e+05,2.105849e+05
1730978,592,20230216,002999.SZ,天禾股份,13.9793,14.0081,13.3753,13.5191,182762.30,174409.003,...,27.34,2.8588,0.2513,0.1982,1.0638,1.0638,3.475920e+04,2.177714e+04,3.267365e+05,2.047051e+05


In [18]:
stock_daily=pd.read_sql("select * from stock_daily where 股票代码>'003000.SZ'",con=conn)
cwzbsj=pd.read_sql("select * from cwzbsj",con=conn)

In [19]:
stock_daily

,index,交易日期,股票代码,股票简称,开盘价,最高价,最低价,收盘价,成交量(手),成交额(千元),...,市盈率(动态),市净率,市销率,市销率(TTM),股息率(%),股息率(TTM)(%),总股本(万股),流通股本(万股),总市值(万元),流通市值(万元)
0,0,20201013,003001.SZ,中岩大地,36.1900,43.4300,36.1900,43.4300,1755.12,7587.603,...,32.47,3.4008,4.1228,3.9052,NaN,NaN,9717.5312,2429.3828,422032.3800,105508.0950
1,1,20201014,003001.SZ,中岩大地,47.7700,47.7700,44.0100,47.7700,109022.37,508904.227,...,35.71,3.7406,4.5348,4.2955,NaN,NaN,9717.5312,2429.3828,464206.4654,116051.6164
2,2,20201015,003001.SZ,中岩大地,46.4400,46.8000,43.0400,43.0900,81268.36,362670.681,...,32.21,3.3741,4.0905,3.8746,NaN,NaN,9717.5312,2429.3828,418728.4194,104682.1049
3,3,20201016,003001.SZ,中岩大地,41.0200,43.8900,40.1100,41.9400,57613.41,239760.235,...,31.35,3.2841,3.9814,3.7712,NaN,NaN,9717.5312,2429.3828,407553.2585,101888.3146
4,4,20201019,003001.SZ,中岩大地,41.1300,41.8000,39.2500,39.4500,47553.71,191779.239,...,29.49,3.0891,3.7450,3.5473,NaN,NaN,9717.5312,2429.3828,383356.6058,95839.1515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132744,301,20230213,430047.BJ,诺思兰德,6161.7118,6395.8655,6161.7118,6317.8143,9090.62,13245.225,...,NaN,18.0001,66.0434,48.3345,NaN,NaN,25771.8203,18592.2790,375495.4218,270889.5050
1132745,302,20230214,430047.BJ,诺思兰德,6330.8228,6365.5122,6287.4610,6365.5122,4324.41,6313.445,...,NaN,18.1360,66.5420,48.6994,NaN,NaN,25771.8203,18592.2790,378330.3220,272934.6557
1132746,303,20230215,430047.BJ,诺思兰德,6339.4952,6408.8740,6339.4952,6348.1675,2639.02,3886.946,...,NaN,18.0865,66.3607,48.5667,NaN,NaN,25771.8203,18592.2790,377299.4492,272190.9646
1132747,304,20230216,430047.BJ,诺思兰德,6326.4866,6391.5293,6183.3927,6222.4183,4853.73,6999.374,...,NaN,17.7283,65.0462,47.6047,NaN,NaN,25771.8203,18592.2790,369825.6213,266799.2037


In [20]:
cwzbsj

,index,股票代码,公告日期,报告期,基本每股收益,稀释每股收益,每股营业总收入,每股营业收入,每股资本公积,每股盈余公积,...,营业收入环比增长率(%)(单季度),营业利润同比增长率(%)(单季度),营业利润环比增长率(%)(单季度),净利润同比增长率(%)(单季度),净利润环比增长率(%)(单季度),归属母公司股东的净利润同比增长率(%)(单季度),归属母公司股东的净利润环比增长率(%)(单季度),净资产同比增长率,研发费用,更新标识
0,85,000001.SZ,20180420,20180331,0.33,0.33,1.6323,1.6323,3.2886,0.6279,...,7.9875,4.1495,64.3500,6.1313,63.4044,6.1313,63.4044,7.5345,None,1
1,86,000001.SZ,20180816,20180630,0.73,0.73,3.3338,3.3338,3.2886,0.6279,...,4.2425,7.2488,2.7260,6.8927,2.7597,6.8927,2.7597,7.8916,None,1
2,87,000001.SZ,20181024,20180930,1.14,1.14,5.0474,5.0474,3.2886,0.6279,...,0.7120,7.9320,4.4681,7.3496,4.5300,7.3496,4.5300,7.8492,None,1
3,88,000001.SZ,20190307,20181231,1.39,1.39,6.7977,6.7977,3.2886,0.6279,...,2.1378,8.9395,-38.2219,8.0773,-38.4246,8.0773,-38.4246,8.1007,None,1
4,89,000001.SZ,20190424,20190331,0.38,0.36,1.8914,1.8914,3.2886,0.6279,...,8.0660,12.6165,69.8647,12.9037,70.7015,12.9037,70.7015,12.3313,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89165,1,873527.BJ,20221028,20210930,NaN,NaN,NaN,NaN,NaN,NaN,...,8.4368,NaN,20.8129,NaN,18.3438,NaN,18.3438,NaN,None,0
89166,2,873527.BJ,20220929,20211231,0.71,0.71,8.6172,8.6172,1.3959,0.3103,...,16.8780,NaN,90.8780,NaN,86.4179,NaN,86.4179,12.5044,13731119.82,1
89167,3,873527.BJ,20220609,20220331,0.09,0.09,1.6523,1.6523,1.3959,0.3103,...,-39.2054,21.8618,-72.9238,22.7886,-71.6256,22.7886,-71.6256,NaN,2749094.0,1
89168,4,873527.BJ,20220810,20220630,0.30,0.30,3.8368,3.8368,1.3959,0.3103,...,32.2054,47.9156,136.8954,45.6926,132.7434,45.6926,132.7434,18.7846,6704634.78,1


In [21]:
stock_daily.columns

Index(['index', '交易日期', '股票代码', '股票简称', '开盘价', '最高价', '最低价', '收盘价', '成交量(手)',
       '成交额(千元)', '换手率(%)', '量比', '市盈率(静态)', '市盈率(TTM)', '市盈率(动态)', '市净率',
       '市销率', '市销率(TTM)', '股息率(%)', '股息率(TTM)(%)', '总股本(万股)', '流通股本(万股)',
       '总市值(万元)', '流通市值(万元)'],
      dtype='object')

In [22]:
z1=stock_daily[stock_daily['股票简称'].isin(["格力电器","比亚迪"])].pivot(index="交易日期",columns="股票简称",values="收盘价").fillna(0)

In [23]:
z1

股票简称
交易日期


In [27]:
stock_daily["date"]=stock_daily["交易日期"].astype("str").astype("datetime64[ns]")

In [29]:
cwzbsj[['股票代码', '公告日期', '报告期']]

,股票代码,公告日期,报告期
0,000001.SZ,20180420,20180331
1,000001.SZ,20180816,20180630
2,000001.SZ,20181024,20180930
3,000001.SZ,20190307,20181231
4,000001.SZ,20190424,20190331
...,...,...,...
89165,873527.BJ,20221028,20210930
89166,873527.BJ,20220929,20211231
89167,873527.BJ,20220609,20220331
89168,873527.BJ,20220810,20220630


In [30]:
print(list(cwzbsj.columns))

['index', '股票代码', '公告日期', '报告期', '基本每股收益', '稀释每股收益', '每股营业总收入', '每股营业收入', '每股资本公积', '每股盈余公积', '每股未分配利润', '非经常性损益', '扣除非经常性损益后的净利润(扣非净利润)', '毛利', '流动比率', '速动比率', '保守速动比率', '存货周转天数', '应收账款周转天数', '存货周转率', '应收账款周转率', '流动资产周转率', '固定资产周转率', '总资产周转率', '经营活动净收益', '价值变动净收益', '利息费用', '折旧与摊销', '息税前利润', '息税折旧摊销前利润', '企业自由现金流量', '股权自由现金流量', '无息流动负债', '无息非流动负债', '带息债务', '净债务', '有形资产', '营运资金', '营运流动资本', '全部投入资本', '留存收益', '期末摊薄每股收益', '每股净资产', '每股经营活动产生的现金流量净额', '每股留存收益', '每股现金流量净额', '每股息税前利润', '每股企业自由现金流量', '每股股东自由现金流量', '销售净利率', '销售毛利率', '销售成本率', '销售期间费用率', '净利润/营业总收入', '销售费用/营业总收入', '管理费用/营业总收入', '财务费用/营业总收入', '资产减值损失/营业总收入', '营业总成本/营业总收入', '营业利润/营业总收入', '息税前利润/营业总收入', '净资产收益率', '加权平均净资产收益率', '净资产收益率(扣除非经常损益)', '总资产报酬率', '总资产净利润', '投入资本回报率', '年化净资产收益率', '年化总资产报酬率', '平均净资产收益率(增发条件)', '经营活动净收益/利润总额', '价值变动净收益/利润总额', '营业外收支净额/利润总额', '所得税/利润总额', '扣除非经常损益后的净利润/净利润', '销售商品提供劳务收到的现金/营业收入', '经营活动产生的现金流量净额/营业收入', '经营活动产生的现金流量净额/经营活动净收益', '资本支出/折旧和摊销', '资产负债率', '权益乘数', '权益乘数(杜邦分析)', '流动资产/总资产', '非流动资产/总资产', '

# 股票数据产品主要功能开发

* 传统的财报数据分析文档

* 分析思路

1. 单股分析，多股对比分析

2. 财务数据分析与行情数据分析

3. 股票估值分析

4. 财务指标的维度：安全性，盈利能力，成长行，管理能力

5. 风险风险

## 单股分析


### 完成单个股票关键财务字段的展示

* @st.cache_data streamlit装饰器的作用 (cache result)

In [31]:
cwzbsj

,index,股票代码,公告日期,报告期,基本每股收益,稀释每股收益,每股营业总收入,每股营业收入,每股资本公积,每股盈余公积,...,营业收入环比增长率(%)(单季度),营业利润同比增长率(%)(单季度),营业利润环比增长率(%)(单季度),净利润同比增长率(%)(单季度),净利润环比增长率(%)(单季度),归属母公司股东的净利润同比增长率(%)(单季度),归属母公司股东的净利润环比增长率(%)(单季度),净资产同比增长率,研发费用,更新标识
0,85,000001.SZ,20180420,20180331,0.33,0.33,1.6323,1.6323,3.2886,0.6279,...,7.9875,4.1495,64.3500,6.1313,63.4044,6.1313,63.4044,7.5345,None,1
1,86,000001.SZ,20180816,20180630,0.73,0.73,3.3338,3.3338,3.2886,0.6279,...,4.2425,7.2488,2.7260,6.8927,2.7597,6.8927,2.7597,7.8916,None,1
2,87,000001.SZ,20181024,20180930,1.14,1.14,5.0474,5.0474,3.2886,0.6279,...,0.7120,7.9320,4.4681,7.3496,4.5300,7.3496,4.5300,7.8492,None,1
3,88,000001.SZ,20190307,20181231,1.39,1.39,6.7977,6.7977,3.2886,0.6279,...,2.1378,8.9395,-38.2219,8.0773,-38.4246,8.0773,-38.4246,8.1007,None,1
4,89,000001.SZ,20190424,20190331,0.38,0.36,1.8914,1.8914,3.2886,0.6279,...,8.0660,12.6165,69.8647,12.9037,70.7015,12.9037,70.7015,12.3313,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89165,1,873527.BJ,20221028,20210930,NaN,NaN,NaN,NaN,NaN,NaN,...,8.4368,NaN,20.8129,NaN,18.3438,NaN,18.3438,NaN,None,0
89166,2,873527.BJ,20220929,20211231,0.71,0.71,8.6172,8.6172,1.3959,0.3103,...,16.8780,NaN,90.8780,NaN,86.4179,NaN,86.4179,12.5044,13731119.82,1
89167,3,873527.BJ,20220609,20220331,0.09,0.09,1.6523,1.6523,1.3959,0.3103,...,-39.2054,21.8618,-72.9238,22.7886,-71.6256,22.7886,-71.6256,NaN,2749094.0,1
89168,4,873527.BJ,20220810,20220630,0.30,0.30,3.8368,3.8368,1.3959,0.3103,...,32.2054,47.9156,136.8954,45.6926,132.7434,45.6926,132.7434,18.7846,6704634.78,1


In [32]:
cwzbsj.set_index("报告期", inplace=True)

In [33]:
cwzbsj.set_index("date", inplace=True)

KeyError: "None of ['date'] are in the columns"

In [34]:
cwzbsj

,index,股票代码,公告日期,基本每股收益,稀释每股收益,每股营业总收入,每股营业收入,每股资本公积,每股盈余公积,每股未分配利润,...,营业收入环比增长率(%)(单季度),营业利润同比增长率(%)(单季度),营业利润环比增长率(%)(单季度),净利润同比增长率(%)(单季度),净利润环比增长率(%)(单季度),归属母公司股东的净利润同比增长率(%)(单季度),归属母公司股东的净利润环比增长率(%)(单季度),净资产同比增长率,研发费用,更新标识
报告期,,,,,,,,,,,,,,,,,,,,,
20180331,85,000001.SZ,20180420,0.33,0.33,1.6323,1.6323,3.2886,0.6279,4.6852,...,7.9875,4.1495,64.3500,6.1313,63.4044,6.1313,63.4044,7.5345,None,1
20180630,86,000001.SZ,20180816,0.73,0.73,3.3338,3.3338,3.2886,0.6279,4.9440,...,4.2425,7.2488,2.7260,6.8927,2.7597,6.8927,2.7597,7.8916,None,1
20180930,87,000001.SZ,20181024,1.14,1.14,5.0474,5.0474,3.2886,0.6279,5.3566,...,0.7120,7.9320,4.4681,7.3496,4.5300,7.3496,4.5300,7.8492,None,1
20181231,88,000001.SZ,20190307,1.39,1.39,6.7977,6.7977,3.2886,0.6279,5.5351,...,2.1378,8.9395,-38.2219,8.0773,-38.4246,8.0773,-38.4246,8.1007,None,1
20190331,89,000001.SZ,20190424,0.38,0.36,1.8914,1.8914,3.2886,0.6279,5.9178,...,8.0660,12.6165,69.8647,12.9037,70.7015,12.9037,70.7015,12.3313,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210930,1,873527.BJ,20221028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.4368,NaN,20.8129,NaN,18.3438,NaN,18.3438,NaN,None,0
20211231,2,873527.BJ,20220929,0.71,0.71,8.6172,8.6172,1.3959,0.3103,1.9818,...,16.8780,NaN,90.8780,NaN,86.4179,NaN,86.4179,12.5044,13731119.82,1
20220331,3,873527.BJ,20220609,0.09,0.09,1.6523,1.6523,1.3959,0.3103,2.0720,...,-39.2054,21.8618,-72.9238,22.7886,-71.6256,22.7886,-71.6256,NaN,2749094.0,1


In [35]:
cwzbsj[cwzbsj["股票代码"]=="000651.SZ"][['扣除非经常性损益后的净利润(扣非净利润)','净资产收益率(扣除非经常损益)','销售毛利率','销售净利率', '企业自由现金流量', '非经常性损益','净债务','每股净资产']]

,扣除非经常性损益后的净利润(扣非净利润),净资产收益率(扣除非经常损益),销售毛利率,销售净利率,企业自由现金流量,非经常性损益,净债务,每股净资产
报告期,,,,,,,,
20180331,4186013203.83,6.1326,30.8578,14.1718,6.714461e+09,1395611192.8,-83447897105.82,11.7892
20180630,12632290055.79,17.5548,30.0265,14.1634,5.249164e+09,173910407.56,-81474385853.23,13.0198
20180930,21345703240.01,28.0347,30.1534,14.2846,1.677785e+09,-227267487.99,-78138676694.04,14.4098
20181231,25580865501.38,32.6033,30.2282,13.3145,1.660761e+10,621922180.04,-90695400586.28,15.1814
20190331,5113678278.15,5.5338,30.5896,14.0518,6.083189e+09,558190762.94,-92446387419.83,15.5409
20190630,13393426690.0,14.5764,31.0230,14.1995,2.536284e+10,356767398.89,-100059497602.02,15.3667
20190930,21565496233.75,22.4501,30.1627,14.3304,3.571139e+10,552000133.27,-112832228293.35,16.7548
20191231,24171511911.32,23.9939,27.5815,12.5293,3.879401e+10,525129457.52,-105982193943.38,18.3109
20200331,1429583505.27,1.2816,17.4892,7.7363,-7.132382e+09,128430033.74,-98881396650.03,18.7747


In [37]:
cwzbsj["企业自由现金流量"]

报告期
20180331             NaN
20180630    3.165969e+10
20180930             NaN
20181231    1.329879e+11
20190331             NaN
                ...     
20210930             NaN
20211231    4.148790e+07
20220331   -5.409728e+07
20220630   -3.822863e+07
20220930   -8.789537e+07
Name: 企业自由现金流量, Length: 89170, dtype: float64

code003000以内的是上证和深证

In [38]:
stock_daily=pd.read_sql("select * from stock_daily where 股票代码<'003000.SZ'",con=conn)

In [39]:
stock_daily["交易日期"]=stock_daily["交易日期"].astype("str").astype("datetime64")
stock_daily.set_index("交易日期",inplace=True)

ValueError: Passing in 'datetime64' dtype with no precision is not allowed. Please pass in 'datetime64[ns]' instead.

In [40]:
stock_daily

,index,交易日期,股票代码,股票简称,开盘价,最高价,最低价,收盘价,成交量(手),成交额(千元),...,市盈率(动态),市净率,市销率,市销率(TTM),股息率(%),股息率(TTM)(%),总股本(万股),流通股本(万股),总市值(万元),流通市值(万元)
0,6323,20180102,000001.SZ,平安银行,1419.2252,1480.8844,1416.0359,1456.4333,2081592.55,2856543.822,...,9.21,1.1871,2.1839,2.2280,1.1533,1.1533,1.717041e+06,1.691799e+06,2.352346e+07,2.317765e+07
1,6324,20180103,000001.SZ,平安银行,1459.6226,1473.4427,1403.2788,1417.0990,2962498.38,4006220.766,...,8.96,1.1550,2.1249,2.1678,1.1853,1.1853,1.717041e+06,1.691799e+06,2.288816e+07,2.255168e+07
2,6325,20180104,000001.SZ,平安银行,1416.0359,1421.3513,1395.8372,1408.5942,1854509.48,2454543.516,...,8.91,1.1481,2.1121,2.1548,1.1925,1.1925,1.717041e+06,1.691799e+06,2.275080e+07,2.241634e+07
3,6326,20180105,000001.SZ,平安银行,1404.3419,1419.2252,1397.9634,1413.9097,1210312.72,1603289.517,...,8.94,1.1524,2.1201,2.1630,1.1880,1.1880,1.717041e+06,1.691799e+06,2.283665e+07,2.250093e+07
4,6327,20180108,000001.SZ,平安银行,1408.5942,1412.8466,1367.1337,1377.7646,2158620.81,2806099.169,...,8.71,1.1230,2.0659,2.1077,1.2191,1.2191,1.717041e+06,1.691799e+06,2.225285e+07,2.192571e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730975,589,20230213,002999.SZ,天禾股份,14.3820,14.6696,14.3676,14.4827,173284.55,174254.521,...,29.29,3.0625,0.2692,0.2123,0.9930,0.9930,3.475920e+04,2.177714e+04,3.500251e+05,2.192958e+05
1730976,590,20230214,002999.SZ,天禾股份,14.5977,14.6121,14.1087,14.1231,250697.29,248112.457,...,28.57,2.9865,0.2626,0.2070,1.0183,1.0183,3.475920e+04,2.177714e+04,3.413353e+05,2.138515e+05
1730977,591,20230215,002999.SZ,天禾股份,14.1375,14.1663,13.8786,13.9074,174541.09,169258.922,...,28.13,2.9409,0.2585,0.2039,1.0341,1.0341,3.475920e+04,2.177714e+04,3.361215e+05,2.105849e+05
1730978,592,20230216,002999.SZ,天禾股份,13.9793,14.0081,13.3753,13.5191,182762.30,174409.003,...,27.34,2.8588,0.2513,0.1982,1.0638,1.0638,3.475920e+04,2.177714e+04,3.267365e+05,2.047051e+05


In [42]:
type(stock_daily["交易日期"].head(1).get())

TypeError: NDFrame.get() missing 1 required positional argument: 'key'

In [43]:
stock_daily.set_index("交易日期",inplace=True)

In [45]:
cwzbsj.set_index("报告期",inplace=True)

KeyError: "None of ['报告期'] are in the columns"

In [53]:
stock_price_gl=stock_daily[stock_daily["股票代码"]=="000651.SZ"][['收盘价', '成交量(手)','量比', '市盈率(静态)', '市盈率(TTM)', '市盈率(动态)', '市净率','市销率', '市销率(TTM)', '股息率(%)', '股息率(TTM)(%)','总市值(万元)', '流通市值(万元)']]

In [54]:
stock_price_gl

,收盘价,成交量(手),量比,市盈率(静态),市盈率(TTM),市盈率(动态),市净率,市销率,市销率(TTM),股息率(%),股息率(TTM)(%),总市值(万元),流通市值(万元)
交易日期,,,,,,,,,,,,,
20180102,6750.5571,587146.42,1.33,17.6131,13.8206,13.18,4.6331,2.5079,1.9862,3.9867,3.9867,2.716102e+07,2.695677e+07
20180103,6825.3141,794304.89,1.73,17.8081,13.9736,13.32,4.6844,2.5357,2.0082,3.9430,3.9430,2.746181e+07,2.725530e+07
20180104,6964.3621,640485.24,1.18,18.1709,14.2583,13.59,4.7798,2.5873,2.0491,3.8643,3.8643,2.802127e+07,2.781055e+07
20180105,7158.7303,781865.80,1.39,18.6780,14.6562,13.97,4.9132,2.6595,2.1063,3.7594,3.7594,2.880332e+07,2.858672e+07
20180108,7151.2546,788630.47,1.24,18.6585,14.6409,13.96,4.9081,2.6567,2.1041,3.7633,3.7633,2.877324e+07,2.855686e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20230213,6539.2425,286385.56,1.27,8.4750,7.5989,8.01,2.0422,1.0404,0.9911,8.4977,8.4977,1.954661e+07,1.939418e+07
20230214,6569.3860,192681.97,0.90,8.5141,7.6339,8.05,2.0516,1.0452,0.9957,8.4587,8.4587,1.963671e+07,1.948358e+07
20230215,6507.2151,167134.17,0.80,8.4335,7.5616,7.97,2.0322,1.0353,0.9862,8.5395,8.5395,1.945088e+07,1.929919e+07


In [56]:
stock_price_gl.index

Index([20180102, 20180103, 20180104, 20180105, 20180108, 20180109, 20180110,
       20180111, 20180112, 20180115,
       ...
       20230206, 20230207, 20230208, 20230209, 20230210, 20230213, 20230214,
       20230215, 20230216, 20230217],
      dtype='int64', name='交易日期', length=1238)

In [51]:
stock_price_gl.index = pd.to_datetime(stock_price_gl.index)

In [52]:
stock_price_gl

,收盘价,成交量(手),量比,市盈率(静态),市盈率(TTM),市盈率(动态),市净率,市销率,市销率(TTM),股息率(%),股息率(TTM)(%),总市值(万元),流通市值(万元)
交易日期,,,,,,,,,,,,,
1970-01-01 00:00:00.020180102,6750.5571,587146.42,1.33,17.6131,13.8206,13.18,4.6331,2.5079,1.9862,3.9867,3.9867,2.716102e+07,2.695677e+07
1970-01-01 00:00:00.020180103,6825.3141,794304.89,1.73,17.8081,13.9736,13.32,4.6844,2.5357,2.0082,3.9430,3.9430,2.746181e+07,2.725530e+07
1970-01-01 00:00:00.020180104,6964.3621,640485.24,1.18,18.1709,14.2583,13.59,4.7798,2.5873,2.0491,3.8643,3.8643,2.802127e+07,2.781055e+07
1970-01-01 00:00:00.020180105,7158.7303,781865.80,1.39,18.6780,14.6562,13.97,4.9132,2.6595,2.1063,3.7594,3.7594,2.880332e+07,2.858672e+07
1970-01-01 00:00:00.020180108,7151.2546,788630.47,1.24,18.6585,14.6409,13.96,4.9081,2.6567,2.1041,3.7633,3.7633,2.877324e+07,2.855686e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:00:00.020230213,6539.2425,286385.56,1.27,8.4750,7.5989,8.01,2.0422,1.0404,0.9911,8.4977,8.4977,1.954661e+07,1.939418e+07
1970-01-01 00:00:00.020230214,6569.3860,192681.97,0.90,8.5141,7.6339,8.05,2.0516,1.0452,0.9957,8.4587,8.4587,1.963671e+07,1.948358e+07
1970-01-01 00:00:00.020230215,6507.2151,167134.17,0.80,8.4335,7.5616,7.97,2.0322,1.0353,0.9862,8.5395,8.5395,1.945088e+07,1.929919e+07


In [57]:
stock_price_gl["交易日期"]

KeyError: '交易日期'

In [58]:
stock_price_gl.index

Index([20180102, 20180103, 20180104, 20180105, 20180108, 20180109, 20180110,
       20180111, 20180112, 20180115,
       ...
       20230206, 20230207, 20230208, 20230209, 20230210, 20230213, 20230214,
       20230215, 20230216, 20230217],
      dtype='int64', name='交易日期', length=1238)

In [65]:
a = 20180102
a = pd.to_datetime((str(a)))

In [68]:
a.strftime("%Y-%m-%d")

'2018-01-02'

In [71]:
# stock_price_gl.index = stock_price_gl.index.strftime("%Y-%m-%d")

AttributeError: 'Index' object has no attribute 'strftime'

## 多股比较分析

* 安全性指标

⑴流动比率
流动比率=流动资产/流动负债，参考值为2，太高说明利用率不够，太低则有偿债风险。

⑵速动比率
速动比率=(流动资产-存货)/流动负债，参考值为1，太高说明利用率不够，太低则有偿债风险。

⑶资产负债率
资产负债率=负债总额/资产总额；参考值为50%。如果总资产利润率高于借款利息率，则越高越好。反之，则不宜过高。

⑷净资产负债率
净资产负债率=负债总额/所有者权益；净资产负债率高的公司，其生意模式属于高风险高回报；净资产负债率低的属于低风险低回报。

⑸利息保障倍数
利息保障倍数=息税前利润(EBIT)/利息费用=(利润总额+财务费用)/利息费用。该指标反应企业支付利息的能力，数值越大，支付利息的能力越强。

⑹还债能力
用“现金及现金等价物/带息负债”来考查公司的还债能力，参考值为1。

* 盈利能力指标

⑴营业利润率
营业利润率=营业利润/营业收入。该指标反映企业收入中有多少能转化成利润，一般来说，越大越好。

⑵销售毛利率
销售毛利率=毛利/营业收入。巴菲特三要素中的重要指标，参考值40%。

⑶销售净利率
销售净利率=净利润(含少数股东损益)/营业收入。巴菲特三要素中的重要指标，参考值5%。

⑷总资产收益率
总资产收益率=净利润/平均总资产。

⑸净资产收益率
净资产收益率=净利润/净资产。巴菲特三要素中最重要的指标。巴菲特曾提出，如果只能看一个指标，那么就看净资产收益率。该指标为长期投资必看指标，投资某公司持有10年以上时，年化收益率基本就是净资产收益的平均值，参考值15%。

* 杜邦分析
净资产收益率
=净利润/净资产
=(净利润/销售收入)×(销售收入/总资产)×(总资产/净资产)
=净利润率×总资产周转率×财务杠杆率
=12.1531×0.6334×3.0834%
杜邦分析通过对净资产收益率的拆分，反应出该公司主要靠什么盈利，要警惕这个主要因素下降，如：
高净利润的消费品公司，要警惕其利润下降或品牌影响力降低；
高总资产周转率的公司为“超市模式”，要警惕其周转率的降低；
高财务杠杆比率的公司为“银行模式”，要警惕杠杆降低、筹资困难等因素。

⑴扣非净利润率
扣非净利润率=扣除非经常性损益后的净利润(扣非净利润)/营业收入。

⑵总资产周转率
总资产周转率=销售收入/总资产。

⑶财务杠杆率
财务杠杆率=总资产/净资产。

* 成长性指标分析

⑴营业收入及增长率

⑵净利润及增长率
—一般来讲，企业可持续发展的正常增长率为8%-15%，太低了成长性不够，太高了不可持续容易“暴雷”。

* 管理水平指标分析

⑴应收账款周转率

⑵存货周转率

⑶固定资产周转率

* 财务风险研判及排雷

1.带息负债率
带息负债率=带息债务/总资产，参考值为60%，如超过60%则较为激进。

2.有形资产/总资产
有形资产/总资产，比重越大，越倾向于重资产公司，资产过重的公司，在出现行业拐点时要警惕。

3.有形资产利润率
有形资产利润率=息税前利润/有形资产，代表有形资产的获利能力，可做判断是否是重资产公司的补充，该指标如果高于当年银行贷款的2倍则属于轻资产。

4.应收账款/总资产
应收账款占比=(应收账款+其他应收+应收票据)/总资产，参考值30%。比例过大，则别人欠款太多，业务不好做，有坏账风险。如比例突然大幅增加，则警惕销售环节是否出现问题。（如果在应收票据中再减去银行承兑，则数据更为准确，有精力的读者可自行查阅相应公司财报。）

5.其他应收账款/总资产
其他应收账款占比=其他应收账款/总资产。其他应收账款占比做为对应收账款占比的补充，其他应收账款突然大幅增加，比起应收账款大幅增加更要引起警惕。

6.货币资金/带息债务
该指标反应公司在逆周期环境下抵御危机的能力。

7.营收费用率
营收费用率=(销售费用+管理费用+财务费用)/营业总收入，指花了多少费用才创造出当前的收入，反应公司管理水平。

8.毛利费用率
毛利费用率=(销售费用+管理费用+财务费用)/毛利润，与营收费用率相似，指花了多少费用才创造目前的利润，反应公司管理水平。小于30%可持续关注，大于70%时警惕。

9.净利润变现率
净利润变现率=经营活动产生的现金流净额/净利润，该指标反应公司赚的钱有多少装进了口袋里。（比如是不是打了白条（应收账款），是不是还了债（高负债率），是不是对外投资了？）参考值为100%，越大越好。
注：现金流分析不适用于金融类或成长类的科技公司。

10.现金流分析

注：现金流分析不适用于金融类或成长类的科技公司。

⑴现金流类型

见图

⑵期末现金及现金等价物余额
期末现金及现金等价物=现金及现金等价物净增加额+期初现金及现金等价物
=(经营活动现金流净额+投资活动现金流净额+筹资活动现金流净额)-汇率变动对现金的影响。

⑶企业自由现金流
企业自由现金流=企业从经营活动赚来的钱-维持企业盈利能力必须再投资的钱。

11.高管增减持统计
近十年内发生的高管增减持情况统计：

12.会计事务所及审计意见
审计意见分为：标准无保留意见、带强调事项段的无保留意见、保留意见、否定意见和无法表示意见。只有“标准的无保留意见”的公司才值得研究，其他的一律排除。另外，警惕频繁更换会计事务所的公司，不排除有严重财务问题。

13.高管人员变更
对于频繁更换高管，特别是财务总监的公司要引起警惕。


* 估值指标分析
1.分红比率
分红比率=分红/净利润=每股分红/每股收益，该指标反应公司把赚到的钱，拿出来多少分红给了股东。

2.常用指标对比
股息率=分红/股价。

市盈率=股价/每股收益=总市值/净利润。——适用于医药消费、食品饮料等弱周期公司，不适于钢铁、化工、有色、科技、生物工程等周期性行业或新兴科技公司

市净率=股价/每股净资产=总市值/净资产。——适用于钢铁、有色、煤炭、银行、化工、石油、大型基建等重资公司或周期性行业。不适用于轻资产、新兴高科技行业。

市销率=股价/每股营业收入=总市值/营业收入。——适用于轻资产、高营收、低利润的成长股，如高科技互联企业。缺点是不反映负债情况（如高杠杆）和营收质量（如降价促销）。

市现率=股价/每股现金流量净额。——适用于辅助验证市盈率的质量，单独使用作用不大。熊市有效，牛市忽略。

企业价值倍数=EV/EBITDA。——适用于重资产行业和现金流高于利润的行业，可过滤高杠杆公司，不适用于固定资产更新快的公司、亏损的公司和控股结构公司。

PEG=市盈率/净利润增长率。——适用于弱周期和业绩增长稳定的公司，如消费品、医药行业，不适用于业绩爆发式或是较大波动的公司，对未来的净利润增长率无法准确测算。

In [72]:
from IPython.display import Image
Image(filename="自由现金流分析.png")

FileNotFoundError: [Errno 2] No such file or directory: '自由现金流分析.png'

In [78]:
import akshare as ak

stock_individual_info_em_df = ak.stock_individual_info_em()

KeyError: '603777'

In [79]:
stock_hy=pd.read_csv(r'G:\fund\qmt_test\B站学习背包\财务指标\tushare_bak_basic_20250216155614.csv')

In [80]:
stock_hy

,trade_date,ts_code,name,industry,area,pe,float_share,total_share,total_assets,liquid_assets,...,bvps,pb,list_date,undp,per_undp,rev_yoy,profit_yoy,gpr,npr,holder_num
0,20250214,302132.SZ,中航成飞,NaN,NaN,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0
1,20250214,603409.SH,汇通控股,汽车配件,安徽,0.00,0.00,0.00,15.39,7.81,...,0.00,0.00,0,5.98,0.00,40.59,10.22,29.36,15.67,0
2,20250214,301173.SZ,毓恬冠佳,汽车配件,上海,0.00,0.00,0.00,22.73,16.44,...,0.00,0.00,0,4.41,0.00,-3.39,6.72,18.16,7.10,0
3,20250214,430556.BJ,雅达股份,电器仪表,广东,81.67,1.18,1.61,5.98,4.65,...,3.16,3.93,20230317,1.22,0.76,-4.77,-36.07,33.56,8.09,4810
4,20250214,002016.SZ,世荣兆业,区域地产,广东,184.37,8.09,8.09,66.56,43.74,...,5.90,1.23,20040708,34.19,4.23,-41.61,-90.20,38.46,5.79,18809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,20250213,300962.SZ,中金辐照,医疗保健,深圳,31.18,2.64,2.64,11.40,4.21,...,3.54,4.26,20210409,3.60,1.36,4.95,-0.64,64.90,36.68,13912
6996,20250213,001236.SZ,弘业期货,多元金融,江苏,456.17,3.26,10.08,120.00,0.00,...,1.84,5.37,20220805,0.78,0.08,82.17,1702.64,0.97,0.83,40409
6997,20250213,002662.SZ,京威股份,汽车配件,北京,13.20,15.00,15.00,53.04,23.10,...,2.85,1.33,20120309,6.77,0.45,-9.42,-7.41,29.63,13.56,42048
6998,20250213,300320.SZ,海达股份,橡胶,江苏,36.56,4.90,6.01,36.47,25.30,...,3.81,2.48,20120601,11.67,1.94,23.09,20.97,18.10,5.18,16890


In [83]:
stock_hy[stock_hy["ts_code"]=="000651.SZ"]["industry"]

3460    家用电器
Name: industry, dtype: object

In [86]:
stock_hy = stock_hy.rename(columns={'ts_code': '股票代码', 'industry': '二级行业', 'name':'股票简称'})

In [88]:
stock_hy = stock_hy.sort_values(by='股票代码')

In [89]:
stock_hy

,trade_date,股票代码,股票简称,二级行业,area,pe,float_share,total_share,total_assets,liquid_assets,...,bvps,pb,list_date,undp,per_undp,rev_yoy,profit_yoy,gpr,npr,holder_num
3529,20250214,000001.SZ,平安银行,银行,深圳,4.23,194.06,194.06,57459.88,0.00,...,21.67,0.53,19910403,2398.99,12.36,-12.58,0.24,42.90,35.61,517695
694,20250214,000002.SZ,万 科Ａ,全国地产,深圳,0.00,97.17,119.31,13571.80,10162.76,...,19.62,0.40,19910129,756.73,6.34,-24.25,-231.73,9.50,-7.46,627280
6147,20250213,000004.SZ,国华网安,软件服务,深圳,0.00,1.26,1.32,3.85,2.87,...,1.21,11.24,19901201,-12.18,-9.20,-5.78,-1.78,72.96,-49.25,42528
3775,20250214,000004.SZ,国华网安,软件服务,深圳,0.00,1.26,1.32,3.85,2.87,...,1.21,11.32,19901201,-12.18,-9.20,-5.78,-1.78,72.96,-49.25,42528
1047,20250214,000006.SZ,深振业Ａ,区域地产,深圳,0.00,13.50,13.50,208.89,166.86,...,4.78,1.47,19920427,24.36,1.80,482.14,-42613.00,7.66,-13.10,68571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5573,20250213,873726.BJ,卓兆点胶,机械基件,江苏,0.00,0.23,0.82,6.33,4.87,...,6.93,3.44,20231019,0.24,0.29,-57.68,-123.03,50.22,-15.89,4003
631,20250214,873726.BJ,卓兆点胶,机械基件,江苏,0.00,0.23,0.82,6.33,4.87,...,6.93,3.36,20231019,0.24,0.29,-57.68,-123.03,50.22,-15.89,4003
6170,20250213,873806.BJ,云星宇,软件服务,北京,82.21,3.01,3.01,35.30,30.90,...,4.75,3.46,20240111,6.66,2.21,-19.76,-47.22,16.49,3.58,8676
5016,20250214,873806.BJ,云星宇,软件服务,北京,85.82,3.01,3.01,35.30,30.90,...,4.75,3.61,20240111,6.66,2.21,-19.76,-47.22,16.49,3.58,8676


In [ ]:
"股票简称","股票代码","二级行业"